In [12]:
import tensorflow as tf

# Prefer tf.keras symbols to avoid Pylance resolving issues with `tensorflow.keras` imports
layers = tf.keras.layers
models = tf.keras.models
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

In [13]:
IMAGE_SIZE = 256
CHANNELS = 3

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    horizontal_flip=True,
    rotation_range=10
)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode='sparse',
    save_to_dir="AugmentedImages"
)

Found 1506 images belonging to 3 classes.


In [14]:
for image_batch, label_batch in train_generator:
    print(image_batch[0])
    break

[[[0.6563785  0.6093197  0.6171628 ]
  [0.6549452  0.6078864  0.6157295 ]
  [0.6535118  0.606453   0.61429614]
  ...
  [0.526043   0.45937634 0.48290575]
  [0.5176471  0.45098042 0.47450984]
  [0.50949365 0.44282696 0.46635637]]

 [[0.6874761  0.6404173  0.6482604 ]
  [0.68734586 0.640287   0.6481301 ]
  [0.6872155  0.6401567  0.6479998 ]
  ...
  [0.52513087 0.45846424 0.48199365]
  [0.5176471  0.45098042 0.47450984]
  [0.5091027  0.44243604 0.46596545]]

 [[0.6829996  0.6359408  0.6437839 ]
  [0.68378145 0.6367226  0.64456576]
  [0.6845633  0.63750446 0.6453476 ]
  ...
  [0.52421874 0.45755208 0.4810815 ]
  [0.5176471  0.45098042 0.47450984]
  [0.50871176 0.44204515 0.46557456]]

 ...

 [[0.56594706 0.5345745  0.57771176]
  [0.54133415 0.5099616  0.55309886]
  [0.528473   0.49710044 0.5402377 ]
  ...
  [0.4496018  0.3986214  0.4260724 ]
  [0.44777754 0.39679715 0.42424813]
  [0.44595328 0.3949729  0.42242387]]

 [[0.565035   0.53366244 0.5767997 ]
  [0.54081297 0.5094404  0.5525777 ]


In [15]:
validatioin_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    horizontal_flip=True
)

validatioin_generator = validatioin_datagen.flow_from_directory(
    'dataset/val',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode='sparse'
)

Found 215 images belonging to 3 classes.


In [16]:
test_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    horizontal_flip=True
)

test_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode='sparse'
)

Found 431 images belonging to 3 classes.


In [17]:
# Build a simple CNN model (define `model` before calling summary)
try:
    num_classes = int(train_generator.num_classes)
except Exception:
    num_classes = 3

model = tf.keras.Sequential(
    [
        layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
 )

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,363 (56.61 MB)

 Trainable params: 14,839,363 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [19]:
1506/32

47.0625

In [20]:
model.fit(
    train_generator,
    steps_per_epoch=47,
    batch_size=32,
    validation_data=validatioin_generator,
    validation_steps=6,
    verbose=1,
    epochs=20
)

Epoch 1/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 129s 3s/step - accuracy: 0.7062 - loss: 0.8595 - val_accuracy: 0.8542 - val_loss: 0.4470
Epoch 2/20
 1/47 ━━━━━━━━━━━━━━━━━━━━ 1:02 1s/step - accuracy: 0.8750 - loss: 0.3326

d:\Work\On\Potato Disease Classification\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


47/47 ━━━━━━━━━━━━━━━━━━━━ 7s 126ms/step - accuracy: 0.8750 - loss: 0.3326 - val_accuracy: 0.8802 - val_loss: 0.3204
Epoch 3/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.8962 - loss: 0.2692 - val_accuracy: 0.9271 - val_loss: 0.1896
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 7s 117ms/step - accuracy: 0.8125 - loss: 0.3192 - val_accuracy: 0.9479 - val_loss: 0.1636
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.9315 - loss: 0.2127 - val_accuracy: 0.8750 - val_loss: 0.2769
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 7s 128ms/step - accuracy: 0.8750 - loss: 0.2509 - val_accuracy: 0.8802 - val_loss: 0.2891
Epoch 7/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.9125 - loss: 0.2151 - val_accuracy: 0.8333 - val_loss: 0.3885
Epoch 8/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 7s 124ms/step - accuracy: 0.8750 - loss: 0.2897 - val_accuracy: 0.8490 - val_loss: 0.3439
Epoch 9/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 0.9383 - loss: 0.1517 - val_accuracy: 0.9323 - val_loss

In [21]:
scores = model.evaluate(test_generator)

14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.8608 - loss: 0.5148
